This script creates all concurrent admin predictors for all first-term observations, and then append them to the lists of term-specific and course-specific predictors, respectively.

In [1]:
import pandas as pd
import numpy as np

In [2]:
concurrent_courses = pd.read_stata("~\\Box Sync\\Clickstream\\data\\first\\concurrent_courses.dta")

In [3]:
avg_grades = pd.read_stata("~\\Box Sync\\Clickstream\\data\\full\\avg_grades.dta")
avg_dict = {}
avg_dict_2 = {}
for j in range(avg_grades.shape[0]):
    course = avg_grades.course.iloc[j]
    strm = avg_grades.strm.iloc[j]
    college = avg_grades.college.iloc[j]
    avg_dict[(course,strm,college)] = avg_grades.avg_grade.iloc[j]
    avg_dict_2[(strm,college)] = avg_grades.avg_grade_2.iloc[j]

In [4]:
avg_g = []
for i in range(concurrent_courses.shape[0]):
    course = concurrent_courses.course.iloc[i]
    strm = concurrent_courses.strm.iloc[i]
    college = concurrent_courses.college.iloc[i]
    try:
        avg_g.append(avg_dict[(course,strm,college)])
    except KeyError:
        avg_g.append(avg_dict_2[(strm,college)])
concurrent_courses.loc[:,'avg_g'] = avg_g

In [5]:
vccsid_sets = []
l = list(np.unique(concurrent_courses.vccsid))
n = len(l) // 500 + 1
for i in range(500):
    vccsid_sets.append(l[(n*i):(n*(i+1))])

In [6]:
vccsid_sets = [l for l in vccsid_sets if len(l) > 0]
len(vccsid_sets)

499

In [7]:
all_parts = []
for i,ll in enumerate(vccsid_sets):
    print(i)
    ll_series = pd.DataFrame({'vccsid':ll})
    part = ll_series.merge(concurrent_courses, how='inner', on=['vccsid'])
    print(part.shape[0])
    all_parts.append(part.copy())
    print("")

0
437

1
449

2
407

3
414

4
445

5
438

6
440

7
441

8
418

9
412

10
397

11
422

12
424

13
414

14
433

15
403

16
439

17
436

18
402

19
447

20
436

21
458

22
434

23
430

24
432

25
436

26
461

27
432

28
447

29
378

30
386

31
409

32
430

33
422

34
421

35
414

36
443

37
412

38
443

39
416

40
402

41
434

42
427

43
419

44
439

45
400

46
417

47
444

48
434

49
438

50
426

51
419

52
453

53
441

54
441

55
420

56
436

57
445

58
424

59
453

60
443

61
392

62
403

63
423

64
451

65
433

66
454

67
423

68
426

69
427

70
423

71
402

72
440

73
435

74
454

75
453

76
429

77
408

78
423

79
440

80
428

81
415

82
446

83
435

84
423

85
453

86
445

87
425

88
444

89
459

90
424

91
398

92
404

93
368

94
413

95
443

96
455

97
430

98
431

99
428

100
448

101
453

102
406

103
423

104
415

105
431

106
418

107
438

108
415

109
421

110
404

111
446

112
402

113
434

114
442

115
457

116
448

117
434

118
450

119
435

120
455

121
455

122
445

123

In [8]:
def compute_concurrent(t):
    new_t = t.groupby(['vccsid','strm','college','course']).agg('sum').reset_index()
    new_t.loc[:,'avg_g_concurrent'] = new_t.avg_g/new_t.credit
    new_t = new_t.drop(['avg_g', 'credit'], axis=1)
    return new_t

In [9]:
part_new_list = []
for a,part in enumerate(all_parts):
    print(a)
    part.loc[:,'avg_g'] = part.avg_g * part.credit
    all_vccsid_strm = part.loc[:,['vccsid','strm']].drop_duplicates()
    n = all_vccsid_strm.shape[0]
    temp_list = []
    for i in range(n):
        part_sub = all_vccsid_strm.iloc[i:(i+1),:].merge(part, on=['vccsid','strm'])
        indx = []
        indx_2 = []
        m = part_sub.shape[0]
        for j in range(m):
            indx.extend([k for k in range(m) if k != j])
            indx_2.extend([j]*(m-1))
        part_sub1 = part_sub.iloc[indx,:].loc[:,['credit', 'avg_g']]
        part_sub1.index = range(m*(m-1))
        part_sub2 = part_sub.iloc[indx_2,:].loc[:,['vccsid','strm','college', 'course']]
        part_sub2.index = range(m*(m-1))
        temp = pd.concat([part_sub2, part_sub1], axis=1)
        temp_list.append(temp)
    part_new = pd.concat(temp_list)
    part_new = compute_concurrent(part_new)
    part_new_list.append(part_new.copy())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [10]:
merged = pd.concat(part_new_list)

In [11]:
merged.describe()

,strm,avg_g_concurrent
count,213208.000000,213208.000000
mean,2200.078515,2.781399
std,6.305766,0.309458
min,2193.000000,0.500000
25%,2194.000000,2.607076
50%,2203.000000,2.748011
75%,2204.000000,2.917811
max,2212.000000,4.000000


In [12]:
target = pd.read_csv("~\\Box Sync\\Clickstream\\data\\first\\course_specific_predictors.csv")

In [13]:
target = target.merge(merged, on = ['vccsid', 'strm', 'college', 'course'], how='left')

In [14]:
target.loc[:,'has_avg_g_concurrent'] = ~pd.isnull(target.avg_g_concurrent)

In [15]:
target.loc[:,'has_avg_g_concurrent'] = target.loc[:,'has_avg_g_concurrent'].astype(int)

In [16]:
target = target.fillna(0)

In [17]:
target.to_csv("~\\Box Sync\\Clickstream\\data\\first\\course_specific_predictors_new.csv", index=False)
target.shape

(213732, 13)

In [18]:
term = pd.read_csv("~\\Box Sync\\Clickstream\\data\\first\\term_specific_predictors.csv")
term.shape

(73920, 4)

In [19]:
new1 = pd.read_stata("~\\Box Sync\\Clickstream\\data\\first\\additional_term_specific.dta")
new2 = pd.read_stata("~\\Box Sync\\Clickstream\\data\\first\\cip_degree.dta")

In [20]:
term = term.merge(new1, on=['vccsid', 'strm'], how='inner').merge(new2, on=['vccsid','strm'], how='inner')
term.shape

(73920, 10)

In [21]:
term.to_csv("~\\Box Sync\\Clickstream\\data\\first\\term_specific_predictors_new.csv", index=False)